In [5]:
def classify_var_diff_sub(task='taskname', trainList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC08','MSC09','MSC10'], predictList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC08','MSC09','MSC10']):
    import pandas as pd
    import numpy as np
    from mvpa2.datasets import dataset_wizard, vstack
    from mvpa2.generators.partition import NFoldPartitioner
    from mvpa2.measures.base import CrossValidation
    from mvpa2.clfs.meta import MappedClassifier
    from mvpa2.clfs.svm import LinearCSVMC
    import reshape
    import scipy.io
    #make an empty df to store accuracy scores from each run
    accTable=[]
    for i in trainList:
        #Load task FC
        taskFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/'+task+'/'+i+'_parcel_corrmat.mat')
        #Load rest
        restFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/'+i+'_parcel_corrmat.mat')
        #Create a training dataset targets 1/0 ==task/rest, chunk=#days
        #taskdf=dataset_wizard(samples=taskFC, targets=1, chunks=range(10))
        #restdf=dataset_wizard(samples=restFC, targets=0, chunks=range(10))
        combine=vstack((taskFC, restFC))
        training=dataset_wizard(samples=combine, targets=([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), chunks=range(20))
        #Linear SVM with cross validation scheme leave one out, output is the accuracy
        cv=CrossValidation(LinearCSVMC(), NFoldPartitioner(), 
                      enable_ca=['stats'],
                      errorfx=lambda p, t: np.mean(p == t))
        #Train the classifier         
        cv.train(training)
        acc_scores_per_task=[]
        #Lets loop through all scenerios for sub and task of interest
        for j in predictList:
            #Load testing set
            test_taskFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/'+task+'/'+j+'_parcel_corrmat.mat')
            #and all rest files
            test_restFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/'+j+'_parcel_corrmat.mat')
            #Create testing dataset
            #testing_taskdf=dataset_wizard(samples=test_taskFC, targets=1, chunks=range(10))
            #testing_restdf=dataset_wizard(samples=test_restFC, targets=0, chunks=range(10))
            #testing=vstack((testing_taskdf, testing_restdf))
            test_combine=vstack((test_taskFC, test_restFC))
            testing=dataset_wizard(samples=test_combine, targets=([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), chunks=range(20))
            #Time to test
            pre=cv(testing)
            #prediction accuracy
            acc=np.mean(pre)
            acc_scores_per_task.append(acc)
        tmp_df=pd.DataFrame({'Test_variables':predictList, i:acc_scores_per_task}).set_index('Test_variables')
        accTable.append(tmp_df)
    accTable=pd.concat(accTable, axis=1)
    accTable.to_csv('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/MVPA/'+task+'_accTable_diff_sub.csv')

In [6]:
taskName=['mem','mixed','motor']
for t in taskName:
    classify_var_diff_sub(t)